#### Import necessary Libraries for data download and manipulation

In [1]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date
import pickle
import re
import numpy as np
import sys
sys.path.append('./scripts')
import tensorflow as tf
label_encoder = LabelEncoder()
import data_download as dd


2023-09-30 01:45:09.875957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 01:45:11.466124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Global Variables

In [2]:
labels = ['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play', 'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs', 'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers']
op_labels = ['op_'+s for s in labels]
labels_2 = ['Date', 'Time', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
labels_old = ['Date', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
yearly_stats_cols = ['Year','Conf','W_overall','L_overall','T_overall','Pct_overall','W_conf','L_conf','T_conf','Pct_conf','SRS','SOS','AP_Pre','AP_High','AP_Post','CFP_High','CFP_Final','Coach','Bowl','Notes']
hist_cols = ['Date', 'Location', 'Opponent', 'Result', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play', 'pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs', 'penalty', 'penalty_yards', 'lost_fumble', 'interceptions', 'turnovers'] 
offfense_cols = hist_cols[:4] + ['of_' + stat for stat in hist_cols[4:]]
defense_cols = ['de_' + stat for stat in hist_cols[4:]]
all_cols = hist_cols + defense_cols
df_cols = all_cols[4:] + ['op_' + s for s in all_cols[4:]]
teams = dd.read_teams()
schedule_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i-schedule.html'
stats_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i.html'
stats_hist_url_base = 'https://www.sports-reference.com/cfb/schools/%s/%i/gamelog/'
yearly_stats_url_base = 'https://www.sports-reference.com/cfb/schools/%s/'
# Get list of all FBS team urls
making_predictions = True

#### Example of schedule request post-processing
* Here we are loading in the 2020 season data for Georgia. We get this using our request and scraping method defined above.

In [3]:
dd.get_team_stats(stats_hist_url_base % ('air-force', 2022))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,2022-09-03,,Northern Iowa,W (48-17),3,6,50.0,109,1,62,...,10.2,2,21,0,23,5,21,2,0,2
1,2022-09-10,,Colorado,W (41-10),1,5,20.0,8,0,70,...,5.9,0,24,1,25,3,25,3,0,3
2,2022-09-16,@,Wyoming,L (14-17),7,14,50.0,101,2,40,...,5.0,5,9,0,14,0,0,0,0,0
3,2022-09-23,,Nevada,W (48-20),1,3,33.3,80,1,75,...,6.9,1,28,1,30,5,50,0,0,0
4,2022-10-01,,Navy,W (13-10),6,8,75.0,156,1,47,...,6.5,5,10,1,16,5,45,2,0,2
5,2022-10-08,@,Utah State,L (27-34),5,12,41.7,95,1,55,...,5.4,4,14,2,20,7,58,1,1,2
6,2022-10-15,@,Nevada-Las Vegas,W (42-7),1,3,33.3,14,0,76,...,5.3,1,25,0,26,1,10,0,0,0
7,2022-10-22,,Boise State,L (14-19),4,9,44.4,96,0,50,...,4.6,4,9,1,14,4,22,1,0,1
8,2022-11-05,N,Army,W (13-7),6,13,46.2,98,0,53,...,4.9,5,13,0,18,5,40,0,1,1
9,2022-11-12,,New Mexico,W (35-3),2,2,100.0,42,1,70,...,6.4,1,22,0,23,6,40,1,0,1


In [4]:
dd.get_yearly_stats(yearly_stats_url_base, 'connecticut')

,Year,Conf,W_overall,L_overall,T_overall,Pct_overall,W_conf,L_conf,T_conf,Pct_conf,SRS,SOS,AP_Pre,AP_High,AP_Post,CFP_High,CFP_Final,Coach,Bowl,Notes
0,2023,Ind,0,4,0,.000,,,,,-11.50,4.00,,,,,,Jim Mora (0-4),,
1,2022,Ind,6,7,0,.462,,,,,-5.78,-3.09,,,,,,Jim Mora (6-7),Myrtle Beach Bowl (L),
2,2021,Ind,1,11,0,.083,,,,,-19.18,-2.59,,,,,,"Randy Edsall (0-2), Lou Spanos (1-9)",,
3,2020,Ind,0,0,0,,,,,,,,,,,,,Randy Edsall (0-0),,
4,2019,American,2,10,0,.167,0,8,0,.000,-14.83,-1.33,,,,,,Randy Edsall (2-10),,
5,2018,American,1,11,0,.083,0,8,0,.000,-18.99,-1.32,,,,,,Randy Edsall (1-11),,
6,2017,American,3,9,0,.250,2,6,0,.250,-12.29,-0.12,,,,,,Randy Edsall (3-9),,
7,2016,American,3,9,0,.250,1,7,0,.125,-13.30,-2.55,,,,,,Bob Diaco (3-9),,
8,2015,American,6,7,0,.462,4,4,0,.500,-3.45,-1.37,,,,,,Bob Diaco (6-7),St. Petersburg Bowl (L),
9,2014,American,2,10,0,.167,1,7,0,.125,-15.72,-3.64,,,,,,Bob Diaco (2-10),,


In [5]:
dd.get_team_schedule('https://www.sports-reference.com/cfb/schools/georgia/2020-schedule.html')

,Date,Time,Day,Team,Location,Opponent,Conference,W/L,Points,Op_points,Wins,Losses,Streak,Notes
0,"Sep 26, 2020",4:00 PM,Sat,(4) Georgia,@,Arkansas,SEC,W,37,10,1,0,W 1,
1,"Oct 3, 2020",7:30 PM,Sat,(4) Georgia,,(7) Auburn,SEC,W,27,6,2,0,W 2,
2,"Oct 10, 2020",3:30 PM,Sat,(3) Georgia,,(14) Tennessee,SEC,W,44,21,3,0,W 3,
3,"Oct 17, 2020",8:00 PM,Sat,(3) Georgia,@,(2) Alabama,SEC,L,24,41,3,1,L 1,
4,"Oct 31, 2020",12:00 PM,Sat,(5) Georgia,@,Kentucky,SEC,W,14,3,4,1,W 1,"Kroger Field - Lexington, Kentucky"
5,"Nov 7, 2020",3:30 PM,Sat,(5) Georgia,N,(8) Florida,SEC,L,28,44,4,2,L 1,"TIAA Bank Field - Jacksonville, Florida"
6,"Nov 21, 2020",7:30 PM,Sat,(13) Georgia,,Mississippi State,SEC,W,31,24,5,2,W 1,
7,"Nov 28, 2020",7:30 PM,Sat,(13) Georgia,@,South Carolina,SEC,W,45,16,6,2,W 2,
8,"Dec 12, 2020",12:00 PM,Sat,(12) Georgia,@,Missouri,SEC,W,49,14,7,2,W 3,
9,"Jan 1, 2021",12:00 PM,Fri,(11) Georgia,N,(6) Cincinnati,American,W,24,21,8,2,W 4,"Peach Bowl (Mercedes-Benz Stadium - Atlanta, G..."


#### Looping functions to collect all data for a year
* We can loop over the request and scrape method in order to get all the data for each team for any given year

In [6]:
all_cols

['Date',
 'Location',
 'Opponent',
 'Result',
 'pass_completion',
 'pass_attempts',
 'pass_completion_percent',
 'pass_yards',
 'pass_td',
 'rush_attempt',
 'rush_yards',
 'avg_rush_yard',
 'rush_td',
 'num_plays',
 'total_yard',
 'yard_per_play',
 'pass_first_down',
 'rush_first_down',
 'penalty_first',
 'first_downs',
 'penalty',
 'penalty_yards',
 'lost_fumble',
 'interceptions',
 'turnovers',
 'de_pass_completion',
 'de_pass_attempts',
 'de_pass_completion_percent',
 'de_pass_yards',
 'de_pass_td',
 'de_rush_attempt',
 'de_rush_yards',
 'de_avg_rush_yard',
 'de_rush_td',
 'de_num_plays',
 'de_total_yard',
 'de_yard_per_play',
 'de_pass_first_down',
 'de_rush_first_down',
 'de_penalty_first',
 'de_first_downs',
 'de_penalty',
 'de_penalty_yards',
 'de_lost_fumble',
 'de_interceptions',
 'de_turnovers']

In [7]:
#dd.download_all_stats()
#dd.download_all_schedules()
#dd.download_all_stats_hist()
#yearly_data = dd.get_all_yearly_stats(teams)
#dd.save_data(yearly_data, './Data/yearly/SRS.pkl')

#### Loading in all downloaded data

In [7]:
stats_data_dict = dd.load_data('./Data/Stats/all_stats.pkl')#dd.load_stats_data(making_predictions=False)
# Contains all the stats for each team for each year of data
# Format: {Year: {'team_name': pd.DataFrame, ...}, ...}
# dataframe: 3 rows x 22 Cols - all numbers - row 1 is offense, row 2 is defense, row 3 is the diff.
# cols of dataframe: ['Games', 'pass_completion', 'pass_attempts', 'pass_completion_percent', 'pass_yards', 'pass_td', 'rush_attempt', 'rush_yards', 'avg_rush_yard', 'rush_td', 'num_plays', 'total_yard', 'yard_per_play','pass_first_down', 'rush_first_down', 'penalty_first', 'first_downs','penalty', 'penalty_yards', 'lost_fumble', 'interceptions','turnovers']
schedule_data_dict = dd.load_stats_data('./Data/Schedules/%i_schedule.pkl', making_predictions=False)
# Contains all the schedules for each team for each year of data
# Format: {Year: {'team_name': pd.DataFrame, ...}, ...}
# dataframe: variable number of rows based on number of games that season
# cols of dataframe: ['Date', 'Day', 'Team', 'Location', 'Opponent', 'Conference', 'W/L', 'Points', 'Op_points', 'Wins', 'Losses', 'Streak', 'Notes']
#hist_data_dict = dd.load_stats_data('./Data/Stats/%i_history.pkl', making_predictions=False)
#yearly_data_dict = dd.load_data('./Data/yearly/SRS.pkl')

In [8]:
#full_dataset = dd.make_dataframe(stats_data_dict, schedule_data_dict)

2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [9]:
#dd.save_data(full_dataset, './Dataset/numerical_dataset_srs.pkl')

In [11]:
print(stats_data_dict[2023]['alabama'])
#print(yearly_data_dict['alabama']) 

  Games pass_completion pass_attempts pass_completion_percent pass_yards  \
0     4            14.5          23.8                    61.1      203.3   
1     4            20.8          34.3                    60.6      203.5   
2                  -6.3         -10.5                    +0.5       -0.2   

  pass_td rush_attempt rush_yards avg_rush_yard rush_td  ... rush_first_down  \
0     1.5         40.5      161.5           4.0     2.0  ...             9.8   
1     1.0         34.5      102.5           3.0     0.5  ...             6.5   
2    +0.5         +6.0      +59.0          +1.0    +1.5  ...            +3.3   

  penalty_first first_downs penalty penalty_yards lost_fumble interceptions  \
0           2.5        19.5     5.8          51.0         0.3           0.8   
1           1.5        16.8     6.5          58.3         0.5           0.8   
2          +1.0        +2.7    -0.7          -7.3        -0.2           0.0   

  turnovers   SRS   SOS  
0       1.0  21.8  11.3  
1    

In [12]:
#dd.save_data(stats_data_dict, './Data/Stats/all_stats.pkl')

In [10]:
full_dataset = dd.load_data('./Dataset/numerical_dataset_srs.pkl')

In [19]:
#full_dataset
value_mapping = {value: idx for idx, value in enumerate(pd.concat([full_dataset['Team_name'], full_dataset['Op_name']]).unique())}
value_mapping
dd.save_data(value_mapping, './Data/Schedules/value_map.pkl')

## Current Prediction code

In [14]:
full_dataset = dd.make_cur_dataframe(stats_data_dict[2023], schedule_data_dict[2023])
print(full_dataset)
dates_to_filter = ['2023-9-28','2023-9-29','2023-9-30','2023-10-1']
dates_to_filter = [s for s in dates_to_filter]
this_weekend = full_dataset[full_dataset['Date'].apply(dd.standardize).isin(dates_to_filter)]
this_weekend = this_weekend.reset_index(drop=True)

'chattanooga'
'holy-cross'
'utah-tech'
'sacred-heart'
'gardner-webb'
'north-alabama'
'merrimack'
'delaware-state'
'campbell'
'abilene-christian'
'howard'
'texas-a&m-commerce'
'wagner'
'kennesaw-state'
'abilene-christian'
'nicholls-state'
'william-&-mary'
            Date  Team  Location  Opponent  Conference Label  Binary_Label  \
0      2023-9-22     0         1        97           7                   0   
1      2023-9-30     0         0        96           7                   0   
2     2023-10-14     0         0       132           7                   0   
3     2023-10-21     0         1        71           1                   0   
4     2023-10-28     0         1        26           7                   0   
...          ...   ...       ...       ...         ...   ...           ...   
1169  2023-10-28   132         1        13           7                   0   
1170   2023-11-3   132         0        26           7                   0   
1171  2023-11-10   132         1        74 

In [15]:
this_weekend

,Date,Team,Location,Opponent,Conference,Label,Binary_Label,pass_completion,pass_attempts,pass_completion_percent,...,op_first_downs,op_penalty,op_penalty_yards,op_lost_fumble,op_interceptions,op_turnovers,op_SRS,op_SOS,Team_name,Op_name
0,2023-9-30,0,0,96,7,,0,2.0,3.0,66.7,...,22.8,5.4,53.8,0.4,1.6,2.0,-3.34,3.26,air-force,san-diego-state
1,2023-9-30,1,0,17,6,,0,24.3,37.0,65.5,...,23.5,6.8,53.8,0.3,1.0,1.3,-12.27,2.48,akron,buffalo
2,2023-9-30,2,1,69,9,,0,14.5,23.8,61.1,...,19.8,5.0,41.3,0.8,1.0,1.8,-0.45,-0.45,alabama,mississippi-state
3,2023-9-30,4,1,54,10,,0,19.8,34.0,58.1,...,19.3,4.3,33.3,1.0,1.3,2.3,-2.00,0.34,appalachian-state,louisiana-monroe
4,2023-9-30,5,0,126,8,,0,24.0,33.8,71.1,...,21.5,5.8,46.8,0.0,1.8,1.8,28.26,4.26,arizona,washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2023-9-30,126,1,5,8,,0,28.3,38.3,73.9,...,16.0,8.0,69.8,0.8,0.0,0.8,1.12,-10.13,washington,arizona
108,2023-9-30,128,1,110,2,,0,13.0,23.0,56.5,...,19.5,6.0,40.0,0.5,1.3,1.8,14.51,0.26,west-virginia,texas-christian
109,2023-9-28,129,0,66,4,,0,26.8,41.8,64.1,...,19.8,6.0,52.3,0.8,0.3,1.0,-2.63,5.17,western-kentucky,middle-tennessee-state
110,2023-9-30,130,0,11,6,,0,16.5,29.3,56.4,...,19.0,4.8,41.5,0.5,0.8,1.3,-9.40,2.60,western-michigan,ball-state


In [16]:
dd.save_data(this_weekend, f'./Data/predictions/{date.today()}.pkl')

In [11]:
print(full_dataset.shape)
missing_rows = full_dataset.loc[full_dataset.isin(['']).any(axis=1)]
print(missing_rows)

(14044, 52)
Empty DataFrame
Columns: [Team, Location, Opponent, Conference, Label, Binary_Label, pass_completion, pass_attempts, pass_completion_percent, pass_yards, pass_td, rush_attempt, rush_yards, avg_rush_yard, rush_td, num_plays, total_yard, yard_per_play, pass_first_down, rush_first_down, penalty_first, first_downs, penalty, penalty_yards, lost_fumble, interceptions, turnovers, SRS, SOS, op_pass_completion, op_pass_attempts, op_pass_completion_percent, op_pass_yards, op_pass_td, op_rush_attempt, op_rush_yards, op_avg_rush_yard, op_rush_td, op_num_plays, op_total_yard, op_yard_per_play, op_pass_first_down, op_rush_first_down, op_penalty_first, op_first_downs, op_penalty, op_penalty_yards, op_lost_fumble, op_interceptions, op_turnovers, op_SRS, op_SOS]
Index: []

[0 rows x 52 columns]


In [ ]:
dd.save_data(this_weekend, './Data/predictions/sep-21-2023.pkl')

In [12]:
#full_dataset['Label'] = full_dataset['Label'].astype('int32')
#label_cols = full_dataset['Date', 'Team']
#full_dataset = full_dataset.drop(['Date', 'Team'], axis=1)
#full_dataset = full_dataset #+ label_cols
full_dataset = dd.obj_to_float(full_dataset)
full_dataset.dtypes

Team                            int32
Location                        int32
Opponent                        int32
Conference                      int32
Label                         float32
Binary_Label                    int16
pass_completion               float32
pass_attempts                 float32
pass_completion_percent       float32
pass_yards                    float32
pass_td                       float32
rush_attempt                  float32
rush_yards                    float32
avg_rush_yard                 float32
rush_td                       float32
num_plays                     float32
total_yard                    float32
yard_per_play                 float32
pass_first_down               float32
rush_first_down               float32
penalty_first                 float32
first_downs                   float32
penalty                       float32
penalty_yards                 float32
lost_fumble                   float32
interceptions                 float32
turnovers   

In [13]:
new_data, scale = dd.normalize_data(full_dataset, './Dataset/MMScaler_srs.pkl')

In [14]:
new_data.describe()

,Label,Binary_Label,Team,Location,Opponent,Conference,pass_completion,pass_attempts,pass_completion_percent,pass_yards,...,op_rush_first_down,op_penalty_first,op_first_downs,op_penalty,op_penalty_yards,op_lost_fumble,op_interceptions,op_turnovers,op_SRS,op_SOS
count,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,...,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000,14044.000000
mean,28.124537,0.499146,0.504018,0.307711,0.504187,0.490224,0.426179,0.466630,0.582241,0.457006,...,0.469947,0.398710,0.477997,0.440121,0.415229,0.332238,0.354466,0.435093,0.520155,0.504777
std,14.134821,0.500017,0.292513,0.312453,0.292524,0.324482,0.122993,0.115225,0.113580,0.133819,...,0.145052,0.148438,0.145101,0.126134,0.136498,0.124964,0.132034,0.134387,0.165343,0.187663
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,0.000000,0.250000,0.000000,0.250000,0.200000,0.360976,0.406199,0.511156,0.376084,...,0.360902,0.285714,0.379679,0.350000,0.316667,0.250000,0.280000,0.354839,0.403421,0.343083
50%,27.000000,0.000000,0.507576,0.500000,0.507576,0.500000,0.424390,0.469821,0.586207,0.455657,...,0.458647,0.400000,0.481283,0.437500,0.408333,0.350000,0.320000,0.419355,0.523667,0.520525
75%,38.000000,1.000000,0.757576,0.500000,0.757576,0.800000,0.495122,0.535073,0.659229,0.531062,...,0.563910,0.485714,0.577540,0.525000,0.502778,0.400000,0.440000,0.516129,0.632287,0.662151
max,82.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
dd.save_data(new_data, './Dataset/scaled_dataset_srs.pkl')